In [ ]:
!pip install kaggle


In [ ]:
!mkdir -p /root/.kaggle
!mv kaggle.json /root/.kaggle/

In [ ]:
!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset

In [ ]:
!unzip face-expression-recognition-dataset.zip

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import glob

# Define the dataset class
class MyDataset(Dataset):
    def __init__(self, transform=None, string="train"):
        self.imgs_path = "images/" + string + "/"
        file_list = glob.glob(self.imgs_path + "*")
        self.data = []
        for class_path in file_list:
            class_name = os.path.basename(class_path)
            for img_path in glob.glob(class_path + "/*.jpg"):
                self.data.append([img_path, class_name])
        self.class_map = {"angry": 0, "disgust": 1, "fear": 2, "happy": 3, "neutral": 4, "sad": 5, "surprise": 6}
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        img = Image.open(img_path)
        class_id = self.class_map[class_name]
        if self.transform:
            img = self.transform(img)
        return img, class_id

# Define transforms for preprocessing the images
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize images
])

# Load train and validation datasets using MyDataset class
train_dataset = MyDataset(transform=transform, string="train")
val_dataset = MyDataset(transform=transform, string="validation")

# Create DataLoader for batching and shuffling
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

class EmotionClassifier(nn.Module):
    def __init__(self, num_classes=7):
        super(EmotionClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(128 * 8 * 8, 512)  # Adjusted input size after flattening
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        #print("Input shape:", x.shape)
        x = torch.relu(self.conv1(x))
        #print("Shape after conv1:", x.shape)
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        #print("Shape after maxpool1:", x.shape)
        x = torch.relu(self.conv2(x))
        #print("Shape after conv2:", x.shape)
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        #print("Shape after maxpool2:", x.shape)
        x = torch.relu(self.conv3(x))
        #print("Shape after conv3:", x.shape)
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        #print("Shape after maxpool3:", x.shape)
        x = x.view(x.size(0), -1)
        #print("Shape after flattening:", x.shape)
        x = torch.relu(self.fc1(x))
        #print("Shape after fc1:", x.shape)
        x = self.fc2(x)
        #print("Shape after fc2:", x.shape)
        return x

# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EmotionClassifier().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")

# Evaluation on validation set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {(correct / total) * 100:.2f}%")


In [ ]:
import random
import matplotlib.pyplot as plt

# Set model to evaluation mode
model.eval()

# Take 5 random images from the validation dataset
random_indices = random.sample(range(len(val_dataset)), 2)
random_images = [val_dataset[i][0] for i in random_indices]
random_labels = [val_dataset[i][1] for i in random_indices]

# Predict emotions for the random images
with torch.no_grad():
    for i, image in enumerate(random_images):
        image = image.unsqueeze(0)  # Add batch dimension
        image = image.to(device)     # Move image to GPU if available
        output = model(image)
        _, predicted_label = torch.max(output, 1)
        predicted_label = predicted_label.item()

        # Map predicted label to emotion
        emotions = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
        predicted_emotion = emotions[predicted_label]

        # Get actual label (emotion)
        actual_label = emotions[random_labels[i]]

        # Display the image and labels
        plt.imshow(random_images[i].permute(1, 2, 0).numpy().squeeze(), cmap='gray')
        plt.title(f"Predicted: {predicted_emotion}\nActual: {actual_label}")
        plt.axis('off')
        plt.show()


Dropout and Reg

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import glob

# Define the dataset class
class MyDataset(Dataset):
    def __init__(self, transform=None, string="train"):
        self.imgs_path = "images/" + string + "/"
        file_list = glob.glob(self.imgs_path + "*")
        self.data = []
        for class_path in file_list:
            class_name = os.path.basename(class_path)
            for img_path in glob.glob(class_path + "/*.jpg"):
                self.data.append([img_path, class_name])
        self.class_map = {"angry": 0, "disgust": 1, "fear": 2, "happy": 3, "neutral": 4, "sad": 5, "surprise": 6}
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        img = Image.open(img_path)
        class_id = self.class_map[class_name]
        if self.transform:
            img = self.transform(img)
        return img, class_id

# Define transforms for preprocessing the images
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize images
])

# Load train and validation datasets using MyDataset class
train_dataset = MyDataset(transform=transform, string="train")
val_dataset = MyDataset(transform=transform, string="validation")

# Create DataLoader for batching and shuffling
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

class EmotionClassifier(nn.Module):
    def __init__(self, num_classes=7):
        super(EmotionClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.dropout2 = nn.Dropout2d(0.25)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.dropout3 = nn.Dropout2d(0.25)
        self.fc1 = nn.Linear(128 * 8 * 8, 512)  # Adjusted input size after flattening
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.dropout1(x)
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv2(x))
        x = self.dropout2(x)
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv3(x))
        x = self.dropout3(x)
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EmotionClassifier().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)  # Apply weight decay

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # Compute training accuracy
        _, predicted_train = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted_train == labels).sum().item()

    # Print training loss and accuracy
    train_loss = running_loss/len(train_loader)
    train_acc = 100 * correct_train / total_train
    print(f"Epoch: [{epoch+1}/{num_epochs}]")
    print(f"\tTraining Loss: {train_loss:.4f} | Accuracy: {train_acc:.2f}%")

    # Evaluation on validation set
    model.eval()
    correct_val = 0
    total_val = 0
    val_running_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item()
            _, predicted_val = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted_val == labels).sum().item()

    # Print validation loss and accuracy
    val_loss = val_running_loss/len(val_loader)
    val_acc = 100 * correct_val / total_val
    print(f"\tValidation Loss: {val_loss:.4f} | Accuracy: {val_acc:.2f}%")


Epoch: [1/10]
	Training Loss: 1.6585 | Accuracy: 33.65%
	Validation Loss: 1.4693 | Accuracy: 43.55%
Epoch: [2/10]
	Training Loss: 1.4407 | Accuracy: 44.37%
	Validation Loss: 1.3391 | Accuracy: 48.50%
Epoch: [3/10]
	Training Loss: 1.3517 | Accuracy: 48.41%
	Validation Loss: 1.2858 | Accuracy: 51.51%
Epoch: [4/10]
	Training Loss: 1.2903 | Accuracy: 50.67%
	Validation Loss: 1.2402 | Accuracy: 52.76%
Epoch: [5/10]
	Training Loss: 1.2441 | Accuracy: 52.52%
	Validation Loss: 1.2115 | Accuracy: 54.50%
Epoch: [6/10]
	Training Loss: 1.2094 | Accuracy: 53.77%
	Validation Loss: 1.2167 | Accuracy: 53.64%
Epoch: [7/10]
	Training Loss: 1.1720 | Accuracy: 55.37%
	Validation Loss: 1.1836 | Accuracy: 55.51%
Epoch: [8/10]
	Training Loss: 1.1446 | Accuracy: 56.68%
	Validation Loss: 1.1756 | Accuracy: 56.52%
Epoch: [9/10]
	Training Loss: 1.1144 | Accuracy: 57.97%
	Validation Loss: 1.1632 | Accuracy: 56.52%
Epoch: [10/10]
	Training Loss: 1.0878 | Accuracy: 58.56%
	Validation Loss: 1.1630 | Accuracy: 56.64%

**Early Stopping**

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import glob

# Define the dataset class
class MyDataset(Dataset):
    def __init__(self, transform=None, string="train"):
        self.imgs_path = "images/" + string + "/"
        file_list = glob.glob(self.imgs_path + "*")
        self.data = []
        for class_path in file_list:
            class_name = os.path.basename(class_path)
            for img_path in glob.glob(class_path + "/*.jpg"):
                self.data.append([img_path, class_name])
        self.class_map = {"angry": 0, "disgust": 1, "fear": 2, "happy": 3, "neutral": 4, "sad": 5, "surprise": 6}
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        img = Image.open(img_path)
        class_id = self.class_map[class_name]
        if self.transform:
            img = self.transform(img)
        return img, class_id

# Define transforms for preprocessing the images
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.Grayscale(num_output_channels=3),  # Convert to grayscale
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

# Load train and validation datasets using MyDataset class
train_dataset = MyDataset(transform=transform, string="train")
val_dataset = MyDataset(transform=transform, string="validation")

# Create DataLoader for batching and shuffling
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

class EmotionClassifier(nn.Module):
    def __init__(self, num_classes=7):
        super(EmotionClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.dropout2 = nn.Dropout2d(0.25)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.dropout3 = nn.Dropout2d(0.25)
        self.fc1 = nn.Linear(128 * 8 * 8, 512)  # Adjusted input size after flattening
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.dropout1(x)
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv2(x))
        x = self.dropout2(x)
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv3(x))
        x = self.dropout3(x)
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EmotionClassifier().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)  # Apply weight decay

# Training loop
# Training loop with early stopping
num_epochs = 30
patience = 3
best_val_loss = float('inf')
best_epoch = 0
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted_train = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted_train == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct_train / total_train

    model.eval()
    correct_val = 0
    total_val = 0
    val_running_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item()
            _, predicted_val = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted_val == labels).sum().item()

    val_loss = val_running_loss / len(val_loader)
    val_acc = 100 * correct_val / total_val

    print(f"Epoch: [{epoch+1}/{num_epochs}]")
    print(f"\tTraining Loss: {train_loss:.4f} | Accuracy: {train_acc:.2f}%")
    print(f"\tValidation Loss: {val_loss:.4f} | Accuracy: {val_acc:.2f}%")

    # Check for early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_epoch = epoch
    else:
        if epoch - best_epoch >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break


Epoch: [1/30]
	Training Loss: 1.6363 | Accuracy: 35.06%
	Validation Loss: 1.4665 | Accuracy: 43.76%
Epoch: [2/30]
	Training Loss: 1.4160 | Accuracy: 45.59%
	Validation Loss: 1.3182 | Accuracy: 49.42%
Epoch: [3/30]
	Training Loss: 1.3266 | Accuracy: 48.98%
	Validation Loss: 1.2628 | Accuracy: 52.14%
Epoch: [4/30]
	Training Loss: 1.2688 | Accuracy: 51.21%
	Validation Loss: 1.2234 | Accuracy: 53.34%
Epoch: [5/30]
	Training Loss: 1.2232 | Accuracy: 53.53%
	Validation Loss: 1.1850 | Accuracy: 55.56%
Epoch: [6/30]
	Training Loss: 1.1897 | Accuracy: 54.98%
	Validation Loss: 1.1746 | Accuracy: 55.35%
Epoch: [7/30]
	Training Loss: 1.1526 | Accuracy: 56.24%
	Validation Loss: 1.1766 | Accuracy: 55.43%
Epoch: [8/30]
	Training Loss: 1.1183 | Accuracy: 57.67%
	Validation Loss: 1.1807 | Accuracy: 55.38%
Epoch: [9/30]
	Training Loss: 1.0924 | Accuracy: 58.84%
	Validation Loss: 1.1588 | Accuracy: 56.69%
Epoch: [10/30]
	Training Loss: 1.0609 | Accuracy: 60.17%
	Validation Loss: 1.1625 | Accuracy: 56.72%

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import glob

# Define the dataset class
class MyDataset(Dataset):
    def __init__(self, transform=None, string="train"):
        self.imgs_path = "images/" + string + "/"
        file_list = glob.glob(self.imgs_path + "*")
        self.data = []
        for class_path in file_list:
            class_name = os.path.basename(class_path)
            for img_path in glob.glob(class_path + "/*.jpg"):
                self.data.append([img_path, class_name])
        self.class_map = {"angry": 0, "disgust": 1, "fear": 2, "happy": 3, "neutral": 4, "sad": 5, "surprise": 6}
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        img = Image.open(img_path)
        class_id = self.class_map[class_name]
        if self.transform:
            img = self.transform(img)
        return img, class_id

# Define transforms for preprocessing the images
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize images
])

# Load train and validation datasets using MyDataset class
train_dataset = MyDataset(transform=transform, string="train")
val_dataset = MyDataset(transform=transform, string="validation")

# Create DataLoader for batching and shuffling
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

class EmotionClassifier(nn.Module):
    def __init__(self, num_classes=7):
        super(EmotionClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.dropout2 = nn.Dropout2d(0.25)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.dropout3 = nn.Dropout2d(0.25)
        self.fc1 = nn.Linear(128 * 8 * 8, 512)  # Adjusted input size after flattening
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.dropout1(x)
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv2(x))
        x = self.dropout2(x)
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv3(x))
        x = self.dropout3(x)
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EmotionClassifier().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)  # Apply weight decay

# Define early stopping parameters
patience = 3
best_val_loss = float('inf')
counter = 0

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(train_loader)

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = (correct / total) * 100

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

    # Implement early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f'Early stopping at epoch {epoch+1}')
            break

#


In [ ]:
import random
import matplotlib.pyplot as plt

# Set model to evaluation mode
model.eval()

# Take 5 random images from the validation dataset
random_indices = random.sample(range(len(val_dataset)), 2)
random_images = [val_dataset[i][0] for i in random_indices]
random_labels = [val_dataset[i][1] for i in random_indices]

# Predict emotions for the random images
with torch.no_grad():
    for i, image in enumerate(random_images):
        image = image.unsqueeze(0)  # Add batch dimension
        image = image.to(device)     # Move image to GPU if available
        output = model(image)
        _, predicted_label = torch.max(output, 1)
        predicted_label = predicted_label.item()

        # Map predicted label to emotion
        emotions = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
        predicted_emotion = emotions[predicted_label]

        # Get actual label (emotion)
        actual_label = emotions[random_labels[i]]

        # Display the image and labels
        plt.imshow(random_images[i].permute(1, 2, 0).numpy().squeeze(), cmap='gray')
        plt.title(f"Predicted: {predicted_emotion}\nActual: {actual_label}")
        plt.axis('off')
        plt.show()


In [ ]:
# Save the trained model
torch.save(model.state_dict(), 'emotion_classifier_model.pth')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Load the saved model
loaded_model = EmotionClassifier().to(device)
loaded_model.load_state_dict(torch.load('emotion_classifier_model.pth'))
loaded_model.eval()

# Preprocess the new image
from torchvision import transforms
from PIL import Image

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

image_path = '/content/IMG_2131.JPG'
image = Image.open(image_path)
image_tensor = transform(image).unsqueeze(0).to(device)

# Make predictions using the loaded model
with torch.no_grad():
    output = loaded_model(image_tensor)
    _, predicted = torch.max(output, 1)
    emotions = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
    predicted_emotion = emotions[predicted.item()]

# Display the image
plt.imshow(image)
plt.title(f'Predicted Emotion: {predicted_emotion}')
plt.axis('off')
plt.show()
